In [66]:
# include("../code/julia/makedata.jl");
L = Array{Float64,2}(CSV.read("../data/sample100000x100.txt", nullable = false, header = false, delim = ' '));

In [100]:
include("../code/julia/QPsubprob.jl");

In [101]:
QPsubprob(L, initialize = "sparse", method = "activeset", warmstart = false)

Dict{String,Any} with 4 entries:
  "timing"      => [0.0511951, 0.0010846, 0.00095883, 0.00103025, 0.00100715, 0…
  "linesearch"  => [2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1…
  "totalQPtime" => 0.0678874
  "numiter"     => 23

In [102]:
QPsubprob(L, initialize = "sparse", method = "activeset", warmstart = true)

Dict{String,Any} with 4 entries:
  "timing"      => [0.000999656, 0.00129793, 0.00127699, 0.00130211, 0.0012645,…
  "linesearch"  => [2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1…
  "totalQPtime" => 0.0182802
  "numiter"     => 23

In [103]:
QPsubprob(L, initialize = "sparse", method = "interiorpoint")

Dict{String,Any} with 4 entries:
  "timing"      => [1.64705, 1.47268, 1.3802, 1.56387, 1.54196, 1.37699, 1.3352…
  "linesearch"  => [2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1…
  "totalQPtime" => 24.7589
  "numiter"     => 23

In [53]:
srand(1)
x = [randn(10^6);3*randn(10^6)];
s = ones(2*10^6); 
L = ash(x,s, mult = 1.05)[3];

In [54]:
QPsubprob(L, initialize = "sparse", warmstart = false)

Dict{String,Any} with 5 entries:
  "timing"      => [0.00666852, 0.000909547, 0.00124374, 0.000965151, 0.0008556…
  "x"           => [0.499933, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0…
  "linesearch"  => [7.0, 5.0, 4.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,…
  "totalQPtime" => 0.0182387
  "numiter"     => 16

In [55]:
QPsubprob(L, initialize = "sparse", warmstart = true)

Dict{String,Any} with 5 entries:
  "timing"      => [0.00691269, 0.000331555, 0.000515658, 0.000268786, 0.000453…
  "x"           => [0.499933, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0…
  "linesearch"  => [7.0, 5.0, 4.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,…
  "totalQPtime" => 0.0152202
  "numiter"     => 16

In [183]:
# solve minimize 0.5y'Hy + y'g subject to y>=0
function mosek_qp(g,H)
    m = length(g)
    
    bkc   = repmat([MSK_BK_LO], m, 1)[:]
    blc   = zeros(m)
    buc   = Inf * ones(m)
    
    bkx   = repmat([MSK_BK_LO], m, 1)[:]
    blx   = zeros(m)
    bux   = Inf * ones(m)
    numvar = length(bkx)
    numcon = length(bkc)
    
    c     = g
    A     = speye(m)
    
    x     = zeros(m)
    
    maketask() do task

        # Append 'numcon' empty constraints.
        # The constraints will initially have no bounds.  
        appendcons(task,numcon)

        # Append 'numvar' variables.
        # The variables will initially be fixed at zero (x=0). 
        appendvars(task,numvar)

        # Set the linear term c_j in the objective.
        putclist(task,[1:numvar;],c)

        # Set the bounds on variable j
        # blx[j] <= x_j <= bux[j] 
        putvarboundslice(task,1,numvar+1,bkx,blx,bux)

        putacolslice(task,1,numvar+1,
                     A.colptr[1:numvar],A.colptr[2:numvar+1],
                     A.rowval,A.nzval)

        putqobj(task,repeat(1:m,outer = m),repeat(1:m,inner = m),H[:])

        putobjsense(task,MSK_OBJECTIVE_SENSE_MINIMIZE)

        # Optimize
        optimize(task)

        prosta = getprosta(task,MSK_SOL_ITR)
        solsta = getsolsta(task,MSK_SOL_ITR)

        x = getxx(task,MSK_SOL_ITR)
    end
    return x
end

mosek_qp (generic function with 1 method)

In [184]:
mosek_qp(g,H)

LoadError: [91mMosek.MosekError(1415, "Only elements in the lower triangle of the quadratic term in the objective should be specified. The element q[0,1] is in the upper triangle.")[39m